# Faiss building blocks: clustering, PCA, quantization

https://github.com/facebookresearch/faiss/wiki/Faiss-building-blocks:-clustering,-PCA,-quantization

Faiss内置一些高效的基本算法: k-means clustering, PCA, PQ encoding/decoding.

> PCA 降维，IVF 是cell，PQ 是压缩

# Clustering
Faiss 提供了一个高效的 k-means 实现. 把 2-D张量(tensor)`x` 聚类成一个向量集合，如下所示:

In [14]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

> 范围(0,1)均匀分布

In [15]:
print('xb.shape', xb.shape)
print('xb[0]', xb[0])
print('------')
print('xq.shape', xq.shape)
print('xq[0]', xq[0])

xb.shape (100000, 64)
xb[0] [0.19151945 0.62210876 0.43772775 0.7853586  0.77997583 0.2725926
 0.27646425 0.8018722  0.95813936 0.87593263 0.35781726 0.5009951
 0.6834629  0.71270204 0.37025076 0.5611962  0.50308317 0.01376845
 0.7728266  0.8826412  0.364886   0.6153962  0.07538124 0.368824
 0.9331401  0.65137815 0.39720258 0.78873014 0.31683612 0.56809866
 0.8691274  0.4361734  0.8021476  0.14376682 0.70426095 0.7045813
 0.21879211 0.92486763 0.44214076 0.90931594 0.05980922 0.18428709
 0.04735528 0.6748809  0.59462476 0.5333102  0.04332406 0.5614331
 0.32966843 0.5029668  0.11189432 0.6071937  0.5659447  0.00676406
 0.6174417  0.9121229  0.7905241  0.99208146 0.95880175 0.7919641
 0.28525096 0.62491673 0.4780938  0.19567518]
------
xq.shape (10000, 64)
xq[0] [0.81432974 0.7409969  0.8915324  0.02642949 0.24954738 0.75948536
 0.33756447 0.0388501  0.06253924 0.04496585 0.6500265  0.14300306
 0.10555115 0.7554373  0.8733019  0.91065574 0.949595   0.4678057
 0.7957018  0.06088004 0.5086

In [16]:
import faiss

ncentroids = 1024
niter = 20
verbose = True
kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose)
kmeans.train(xb)

493312.94

In [17]:
kmeans.centroids

array([[79.03894   ,  0.34537554,  0.53714687, ...,  0.59079725,
         0.4880821 ,  0.4595067 ],
       [42.34449   ,  0.5623033 ,  0.46093923, ...,  0.48434725,
         0.59289134,  0.56627816],
       [36.749817  ,  0.6352938 ,  0.43645158, ...,  0.6142587 ,
         0.5114158 ,  0.52662617],
       ...,
       [18.414587  ,  0.74792314,  0.4864043 , ...,  0.5047074 ,
         0.48473197,  0.47106358],
       [67.36665   ,  0.44266176,  0.34262112, ...,  0.5168862 ,
         0.5216904 ,  0.59634876],
       [51.232037  ,  0.39631492,  0.5089325 , ...,  0.47541136,
         0.5647706 ,  0.5449215 ]], dtype=float32)

结果centroids位于`kmeans.centroids`.

In [18]:
kmeans.centroids.shape

(1024, 64)

沿着每次迭代的目标函数值保存在变量kmeans.obj中(total square error in case of k-means)。

In [19]:
len(kmeans.obj)

20

In [20]:
kmeans.obj

array([804842.8 , 506015.06, 499761.9 , 497422.06, 496135.38, 495346.94,
       494836.62, 494480.78, 494210.84, 494013.75, 493859.06, 493735.47,
       493647.5 , 493568.6 , 493505.4 , 493452.9 , 493410.2 , 493369.8 ,
       493339.38, 493312.94], dtype=float32)

### Assignment
kmeans完成训练后，若想计算从向量集合`x`到聚类centroids的映射, 可以:

In [21]:
x=xq[:5]
D, I = kmeans.index.search(x, 1)

In [22]:
print(I)
print(D)

[[313]
 [ 51]
 [309]
 [ 51]
 [753]]
[[5.9389877]
 [5.1133347]
 [5.9665318]
 [5.5396433]
 [4.9620514]]


- `I`中返回每个向量最近的
- `D`是squared L2 distances

反向操作, 例如，把centroids作为查询向量集合，查找其最近的15个点, 如下:

In [23]:
index = faiss.IndexFlatL2(d)
index.add(xb)
D, I = index.search (kmeans.centroids, 15)
print(I.shape)
print(I)

(1024, 15)
[[79083 78056 78305 ... 78937 79683 78890]
 [42086 41925 41587 ... 41972 42339 41297]
 [36808 36129 35861 ... 36321 36296 35554]
 ...
 [18369 18180 18085 ... 17047 17510 18735]
 [66867 67113 67147 ... 66796 66963 66965]
 [50521 50623 51618 ... 51415 50469 51024]]


`I`的大小为`(ncentroids, 15)`, 其包含每个centroid的最近邻.

### Clustering on the GPU
Clustering on one or several GPUs requires to adapt the indexing object a bit.

# Computing a PCA
让我们把 40D vectors 降到 10D.

In [24]:
# random training data 
mt = np.random.rand(1000, 40).astype('float32')
mat = faiss.PCAMatrix (40, 10)
mat.train(mt)
assert mat.is_trained
tr = mat.apply_py(mt)
# print this to show that the magnitude of tr's columns is decreasing
print((tr ** 2).sum(0))

[116.01529  115.3129   108.345406 107.58896  105.79919  101.523346
 100.9948    98.78388   98.579926  96.67803 ]


Note that in C++, apply_py is replaced with apply (apply is a reserved word in Python).

### PQ encoding / decoding

PQ可用于对向量集合进行encode或decode成code：

In [28]:
d = 32  # data dimension
cs = 4  # code size (bytes)

# train set 
nt = 10000
xt = np.random.rand(nt, d).astype('float32')
print(xt.shape)

# dataset to encode (could be same as train)
n = 20000
x = np.random.rand(n, d).astype('float32')
print(x.shape)

pq = faiss.ProductQuantizer(d, cs, 8)
pq.train(xt)

# encode 
codes = pq.compute_codes(x)

# decode
x2 = pq.decode(codes)

# compute reconstruction error
avg_relative_error = ((x - x2)**2).sum() / (x ** 2).sum()

(10000, 32)
(20000, 32)
(20000, 32)
(20000, 32)
0.066420175
[0.0426398  0.9664037  0.25412124 0.6106941  0.00175703 0.17412935
 0.4433144  0.5724529  0.7539824  0.24685942 0.1142808  0.62381387
 0.8138859  0.91849566 0.87104326 0.32612333 0.7128344  0.40535256
 0.66950816 0.7835837  0.1038271  0.76632565 0.44646704 0.12921831
 0.04432254 0.19473802 0.8664002  0.14597404 0.44283685 0.19634691
 0.5824564  0.9884271 ]
[0.16367587 0.8208272  0.31386742 0.892965   0.2992378  0.18357536
 0.3267108  0.49296647 0.85934895 0.15182856 0.2489067  0.33565524
 0.699809   0.76985663 0.7694404  0.33212703 0.8214057  0.3046869
 0.81794983 0.7870971  0.23961887 0.80900246 0.21303514 0.20706078
 0.2250608  0.15145507 0.77891916 0.28949076 0.29050148 0.19931999
 0.23239535 0.7810108 ]


In [ ]:
print(x.shape)
print(x2.shape)
print(avg_relative_error)
print(x[0])
print(x2[0])

A scalar quantizer works similarly:

In [18]:
d = 32  # data dimension

# train set 
nt = 10000
xt = np.random.rand(nt, d).astype('float32')

# dataset to encode (could be same as train)
n = 20000
x = np.random.rand(n, d).astype('float32')

# QT_8bit allocates 8 bits per dimension (QT_4bit also works)
sq = faiss.ScalarQuantizer(d, faiss.ScalarQuantizer.QT_8bit)
sq.train(xt)

# encode 
codes = sq.compute_codes(x)

# decode
x2 = sq.decode(codes)

# compute reconstruction error
avg_relative_error = ((x - x2)**2).sum() / (x ** 2).sum()

In [27]:
print(avg_relative_error)

0.06621826


# 扩展

### PQ算法
深度学习技术被广泛用于图像识别、语音识别、自然语言处理等领域，把每个实体（图像、语音、文本）转换为对应的embedding向量，一般，相似的实体转换得到的embedding向量也是相似的。对于相似搜索问题，最简单的想法是暴力穷举法，如果全部实体的个数是，是千万量级甚至是上亿的规模，每个实体对应的向量是，那么当要从这个实体集合中寻找某个实体的相似实体，暴力穷举的计算复杂度是，这是一个非常大的计算量，该方法显然不可取。所以对大数据量下高维度数据的相似搜索场景，我们就需要一些高效的相似搜索技术，而PQ就是其中一类方法。

Product Quantizer ，乘积量化器，是一种压缩表达方法，这里的乘积，是指笛卡尔积（cartesian product），把原来的向量空间分解为若干个低维向量空间的笛卡尔积，并对分解得到的低维向量空间分别做量化（quantization）。这样每个向量就能由多个低维空间的量化code组合表示。

乘积量化就是一种编码方法。当图像数据库过于庞大时，直接存储所有的数据并且做相似性度量检索是不现实的。为了能够减少存储开销，提高检索效率，必须对原始数据做瘦身。这其中包括提取图像的特征向量和对提取出的特征向量进行编码。

因为我们可能并不需要图像中的所有信息就可以判断它是否是我们的目标图像。比如，我们想要知道一副图中是否有猫，我们不用关心这只猫的颜色、姿态、位置等信息，我们只需要在图中找到猫的特征就可以证明猫的存在。而这些人类视觉的特征转换成机器语言就是特征向量。提取图像中的特征向量有很多成熟的算法，如SIFT、SURF、ORB、CNN的feature-map等。有很多现成的图像检索算法都是基于SIFT，所以建议如果有时间可以认真学习一下并且亲自动手实现，而不只是调用现成的工具。传统的图像检索方法，如VLAD、BOF、FV等是针对小数据集的，所以它们基本上没有针对大数据集的编码操作，而为了能够实现海量数据的检索工作，避免枚举计算，乘积量化（product quantization）是一个很理想的方法。哈希算法也是一个很不错的算法，但是空间复杂度过高。

考虑一个场景，从100万张图像中提取出了20亿的SIFT描述子用来建数据库。如果没用对这些数据进行编码操作，首先这些向量的存储就是一个大问题，虽然已经比直接保存图像小了很多，但数据量仍然很庞大，之后是查询一条SIFT向量需要计算20亿次！！！！！！这是无法用于实际生产的。乘积量化兼顾空间复杂度和时间复杂度，在两者之间取得了很好的平衡，既可以保证不用存储海量的数据，又保证了检索的速度。

In [8]:
import numpy as np
from sklearn.cluster import KMeans

class PQ:
    def __init__(self, K, L):
        self.dim = L
        self.k = K
    def cut(self, data):
        num = len(data)
        allData = []
        for i in range(num):
            Data = []
            for j in range(int(self.dim / self.k)):
                Data.append(data[i][j*int(self.dim/self.k):(j+1)*int(self.dim/self.k)])
            allData.append(Data)
        return allData
    def kmeans(self, data, K):
        estimator = KMeans(n_clusters=K)
        label = []
        centroids = []
        newdata = np.zeros([len(data), self.k])
        a = np.reshape(data, [len(data), 3, 3])
        # print(data)
        for i in range(self.k):
            fitdata = a[0][i]
            # print(fitdata)
            for w in range(1, len(data)):
                fitdata = np.row_stack((fitdata, a[w][i]))
            estimator.fit(fitdata)
            label.append(estimator.labels_)
            centroids.append(estimator.cluster_centers_)
        for j in range(self.k):
            for k in range(len(data)):
                newdata[k][j] = label[j][k]
        return label, centroids, newdata
    def compute(self, N, query, data):
        dis = np.zeros(len(data))
        for n in range(len(data)):
            for i in range(N):
                if query[i] == data[n][i]:
                    dis[n] = dis[n]
                else:
                    dis[n] = dis[n]+1
        # 按相似度从大到小返回标号
        index = dis.argsort()
        return dis, index
 
 
if __name__ == '__main__':
    data = np.random.randint(0, 3, size=(9, 9))
    pq = PQ(3, 9)
    result = pq.cut(data)
    result2, result3, newdata = pq.kmeans(result, 3)
    print('result2', result2)
    print('result3', result3)
    print('newdata', newdata)
    result4, index = pq.compute(3, newdata[0], newdata)
    print('result4', result4)
    print('index', index)

result2 [array([0, 0, 2, 0, 1, 0, 2, 0, 1], dtype=int32), array([0, 2, 1, 1, 0, 0, 0, 0, 2], dtype=int32), array([0, 0, 2, 0, 0, 2, 1, 1, 1], dtype=int32)]
result3 [array([[1. , 1.2, 1.8],
       [0. , 2. , 0. ],
       [2. , 1. , 0. ]]), array([[0.8, 0.4, 2. ],
       [0. , 2. , 0.5],
       [2. , 0.5, 0.5]]), array([[0.        , 0.75      , 1.        ],
       [1.33333333, 1.33333333, 2.        ],
       [1.5       , 0.        , 0.5       ]])]
newdata [[0. 0. 0.]
 [0. 2. 0.]
 [2. 1. 2.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 2.]
 [2. 0. 1.]
 [0. 0. 1.]
 [1. 2. 1.]]
result4 [0. 1. 3. 1. 1. 1. 2. 1. 3.]
index [0 1 3 4 5 7 6 2 8]
